#### Semantic Chunking for Document Processing

This code implements a semantic chunking approach for processing and retrieving information from PDF documents, first proposed by Greg Kamradt and subsequently implemented in LangChain. Unlike traditional methods that split text based on fixed character or word counts, semantic chunking aims to create more meaningful and context-aware text segments.

Traditional text splitting methods often break documents at arbitrary points, potentially disrupting the flow of information and context. Semantic chunking addresses this issue by attempting to split text at more natural breakpoints, preserving semantic coherence within each chunk.

In [3]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from utility import encode_pdf, show_context, retrieve_context_per_question
from langchain_core.output_parsers import StrOutputParser
from typing import List
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_community.docstore.in_memory import InMemoryDocstore
from tqdm import tqdm
from langchain.vectorstores import Chroma, FAISS
import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from utility import replace_t_with_space
from langchain_experimental.text_splitter import SemanticChunker

In [2]:
file_path = "data/Understanding_Climate_Change.pdf"

In [4]:
#Load the Pdf file 
loader=PyPDFLoader(file_path)
docs=loader.load()

In [8]:
text_data = " ".join([doc.page_content for doc in docs])

In [9]:
#Embeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=90
)
docs = text_splitter.create_documents([text_data])

In [13]:
vectorstore = FAISS.from_documents(docs,embeddings)

In [14]:
retriever = vectorstore.as_retriever(search_kwargs={'k':2})

In [15]:
## Test the retriever 
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, retriever)
show_context(context)

Context 1:
These effects include: 
Rising Temperatures 
Global temperatures have risen by about 1.2 degrees Celsius (2.2 degrees Fahrenheit) since 
the late 19th century. This warming is not uniform, with some regions experiencing more 
significant increases than others. Heatwaves 
Heatwaves are becoming more frequent and severe, posing risks to human health, agriculture, 
and infrastructure. Cities are particularly vulnerable due to the "urban heat island" effect. Heatwaves can lead to heat-related illnesses and exacerbate existing health conditions. Changing Seasons 
Climate change is altering the timing and length of seasons, affecting ecosystems and human 
activities. For example, spring is arriving earlier, and winters are becoming shorter and 
milder in many regions. This shift disrupts plant and animal life cycles and agricultural 
practices. Melting Ice and Rising Sea Levels 
Warmer temperatures are causing polar ice caps and glaciers to melt, contributing to rising 
sea levels